<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

Author: Mariya Mansurova, Analyst & developer in Yandex.Metrics team. Translated by Ivan Zakharov, ML enthusiast. <br>This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center> Assignment #9 (demo)
## <center> Time series analysis

**Same assignment as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a9-demo-time-series-analysis) + [solution](https://www.kaggle.com/kashnitsky/a9-demo-time-series-analysis-solution).**

**Fill cells marked with "Your code here" and submit your answers to the questions through the [web form](https://docs.google.com/forms/d/1UYQ_WYSpsV3VSlZAzhSN_YXmyjV7YlTP8EYMg8M8SoM/edit).**

In [24]:
import warnings

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

warnings.filterwarnings("ignore")
import pandas as pd
from plotly import __version__
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import numpy as np
import statsmodels.api as sm
from tqdm import tqdm
from scipy.stats import boxcox
from scipy.special import inv_boxcox

init_notebook_mode(connected=True)


def plotly_df(df, title=""):
    data = []

    for column in df.columns:
        trace = go.Scatter(x=df.index, y=df[column], mode="lines", name=column)
        data.append(trace)

    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

## Data preparation

In [4]:
df = pd.read_csv("../../data/wiki_machine_learning.csv", sep=" ")
df = df[df["count"] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [5]:
df.shape

(383, 7)

## Predicting with FB Prophet
We will train at first 5 months and predict the number of trips for June.

In [6]:
df.date = pd.to_datetime(df.date)

In [7]:
plotly_df(df.set_index("date")[["count"]])

In [8]:
from prophet import Prophet

In [9]:
predictions = 30

df = df[["date", "count"]]
df.columns = ["ds", "y"]
df.tail()

,ds,y
382,2016-01-16,1644
381,2016-01-17,1836
376,2016-01-18,2983
375,2016-01-19,3389
372,2016-01-20,3559


Create train_df and leave space for N predictions.

In [10]:
train_df = df[:-predictions]

Train the Prophet on this training dataset

In [11]:
p = Prophet()
p.fit(train_df)

15:01:07 - cmdstanpy - INFO - Chain [1] start processing
15:01:07 - cmdstanpy - INFO - Chain [1] done processing


Form dataframe with future dates that we want to predict

In [12]:
future_df = p.make_future_dataframe(periods=predictions, include_history=False)
future_df.tail(5)

,ds
25,2016-01-16
26,2016-01-17
27,2016-01-18
28,2016-01-19
29,2016-01-20


Predict the selected dates

In [13]:
pred = p.predict(future_df)
pred.tail(5)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
25,2016-01-16,2975.100998,1719.094481,2512.377196,2955.735426,2993.385888,-862.545964,-862.545964,-862.545964,-862.545964,-862.545964,-862.545964,0.0,0.0,0.0,2112.555034
26,2016-01-17,2980.432400,1851.459654,2678.813473,2959.883906,2999.885156,-721.525739,-721.525739,-721.525739,-721.525739,-721.525739,-721.525739,0.0,0.0,0.0,2258.906661
27,2016-01-18,2985.763803,2884.258122,3685.356890,2963.641188,3006.648225,281.375854,281.375854,281.375854,281.375854,281.375854,281.375854,0.0,0.0,0.0,3267.139657
28,2016-01-19,2991.095205,3129.777481,3949.868512,2967.328664,3013.615439,542.213022,542.213022,542.213022,542.213022,542.213022,542.213022,0.0,0.0,0.0,3533.308227
29,2016-01-20,2996.426608,3052.914018,3817.474947,2970.764510,3020.384732,426.073193,426.073193,426.073193,426.073193,426.073193,426.073193,0.0,0.0,0.0,3422.499801


View the prediction for January 20th

In [14]:
round(pred[pred["ds"] == "2016-01-20"]["yhat"])

29    3422.0
Name: yhat, dtype: float64

**<font color='red'>Question 1:</font>** What is the prediction of the number of views of the wiki page on January 20? Round to the nearest integer.

- 4947
- `3426`
- 5229
- 2744

Get actual y values for the predicted time period.

In [15]:
y_true = df[-predictions:]["y"]
y_pred = pred["yhat"]

y_true.tail(5)
y_pred.tail(5)

25    2112.555034
26    2258.906661
27    3267.139657
28    3533.308227
29    3422.499801
Name: yhat, dtype: float64

Estimate the quality of the prediction with the last 30 points.

In [16]:
mape = mean_absolute_percentage_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print("MAPE:", mape * 100, '%')
print("MAE:", mae)

MAPE: 34.35047504217826 %
MAE: 596.7308244328995


**<font color='red'>Question 2:</font> What is MAPE equal to?**

- `34.5`
- 42.42
- 5.39
- 65.91

**<font color='red'>Question 3:</font> What is MAE equal to?**

- 355
- 4007
- `600`
- 903

## Predicting with ARIMA

In [19]:
sm.tsa.adfuller(train_df["y"])[1]

np.float64(0.10739219627612018)

If the p-value >= 0.5, the time series is not stationary.

**<font color='red'>Question 4:</font> Let's verify the stationarity of the series using the Dickey-Fuller test. Is the series stationary? What is the p-value?**

- Series is stationary, p_value = 0.107
- `Series is not stationary, p_value = 0.107`
- Series is stationary, p_value = 0.001
- Series is not stationary, p_value = 0.001

In [28]:
def createSarimaxModel(y, pdq, seasonal_pdq):
    best_aic = float("inf")
    best_params = None

    for param in tqdm(pdq):
        for seasonal_param in seasonal_pdq:
            try:
                sarimax = sm.tsa.statespace.SARIMAX(
                    y,
                    order=param,
                    seasonal_order=seasonal_param,
                    enforce_stationarity=False,
                    enforce_invertibility=False,
                )
                result = sarimax.fit(disp=False)

                if result.aic < best_aic:
                    best_aic = result.aic
                    best_params = (param, seasonal_param)
            except:
                continue

    return best_aic, best_params

In [20]:
from itertools import product

p = d = Q = q = D = range(0, 3)
P = range(0, 4)
s = 7

pdq = list(product(p, d, q))
seasonal_pdq = list(product(P, D, Q, [s]))

In [ ]:
%time

aic, params = createSarimaxModel(train_df["y"], pdq, seasonal_pdq)

print(f"Best SARIMAX params: order={params[0]}, seasonal_order={params[1]}, AIC={aic:.2f}")

Received: "Best SARIMAX params: order=(2, 0, 1), seasonal_order=(3, 2, 1, 7), AIC=4624.95"

Let's try to improve this AIC using Box-Cox transformation.

In [25]:
y_transformed, fitted_lambda = boxcox(train_df["y"])
train_df["y_boxcox"] = y_transformed

In [26]:
best_aic = float("inf")
best_params = None

%time

best_aic, best_params = createSarimaxModel(train_df["y_boxcox"], pdq, seasonal_pdq)

print(f"Best SARIMAX params: order={best_params[0]}, seasonal_order={best_params[1]}, AIC={best_aic:.2f}")

CPU times: total: 0 ns
Wall time: 0 ns


100%|██████████| 27/27 [16:11<00:00, 36.00s/it]

Best SARIMAX params: order=(2, 1, 1), seasonal_order=(3, 2, 1, 7), AIC=3280.80


Received: "Best SARIMAX params: order=(2, 1, 1), seasonal_order=(3, 2, 1, 7), AIC=3280.80"

**Next, we turn to the construction of the SARIMAX model (`sm.tsa.statespace.SARIMAX`).<br> <font color='red'>Question 5:</font> What parameters are the best for the model according to the `AIC` criterion?**

- D = 1, d = 0, Q = 0, q = 2, P = 3, p = 1
- D = 2, d = 1, Q = 1, q = 2, P = 3, p = 1
- D = 1, d = 1, Q = 1, q = 2, P = 3, p = 1
- D = 0, d = 0, Q = 0, q = 2, P = 3, p = 1

Since this answer does not exist, let's check each of the provided answers AIC and compare.

In [ ]:
aics = [
    createSarimaxModel(train_df["y_boxcox"], [(1, 0, 2)], [(3, 1, 2, 7)])[0],
    createSarimaxModel(train_df["y_boxcox"], [(1, 1, 2)], [(3, 2, 2, 7)])[0],
    createSarimaxModel(train_df["y_boxcox"], [(1, 1, 2)], [(3, 1, 2, 7)])[0],
    createSarimaxModel(train_df["y_boxcox"], [(1, 0, 2)], [(3, 0, 2, 7)])[0],
]

valid_aics = [aic for aic in aics if aic is not None]

min_aic = min(valid_aics)
print("Min AIC from answers:", min_aic)

100%|██████████| 1/1 [00:00<00:00,  1.02it/s]

[np.float64(3338.230403861751), np.float64(3297.9792443973824), np.float64(3339.4428416521687), np.float64(3488.8321172692786)]
Min AIC from answers: 3297.9792443973824


Received: "Min AIC from answers: 3297.9792443973824". While our min achieved AIC was 3280. Therefore, a better min AIC was found than using the parameters from the answers.

The valid parameters are: order=(2, 1, 1), seasonal_order=(3, 2, 1, 7)